In [ ]:
# packages
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import datetime
import shapely

# personal packages
import utils.geometry

# warnings
import warnings
warnings.filterwarnings("ignore")

# Load and postprocess Landsat fronts

In [2]:
data_root = "D:/OneDrive/Documents/Cours/4A/SFE/data/"
fronts_root = "D:/OneDrive/Documents/Cours/4A/SFE/calving_fronts"
landsat_fronts = gpd.read_file(os.path.join(fronts_root, "Landsat_MSS_fronts_RGI.shp"))
landsat_fronts.columns = ["date", "image", "mission", "timestamp", "RGI60ID", "geometry"]
landsat_fronts = landsat_fronts.drop(columns=["date"])
landsat_fronts.head()

,image,mission,timestamp,RGI60ID,geometry
0,LM02_L1GS_245002_19760505_20200907_02_T2,Landsat 2 MSS,1976-05-05,RGI60-07.00791,"LINESTRING (12.38191 79.53251, 12.38144 79.531..."
1,LM02_L1GS_245002_19760505_20200907_02_T2,Landsat 2 MSS,1976-05-05,RGI60-07.00777,"LINESTRING (12.38935 79.54895, 12.38853 79.549..."
2,LM02_L1GS_245002_19760505_20200907_02_T2,Landsat 2 MSS,1976-05-05,RGI60-07.00775,"LINESTRING (12.32881 79.58411, 12.32937 79.585..."
3,LM02_L1GS_245002_19760505_20200907_02_T2,Landsat 2 MSS,1976-05-05,RGI60-07.00794,"LINESTRING (12.29071 79.65163, 12.28778 79.651..."
4,LM02_L1GS_245002_19760505_20200907_02_T2,Landsat 2 MSS,1976-05-05,RGI60-07.00790,"LINESTRING (12.1169 79.65285, 12.11207 79.6529..."


In [3]:
# add RGIv7 indexes
RGI_link = pd.read_csv(os.path.join(data_root, "Randolph_glacier_inventory/links_v6_v7.csv"))
landsat_fronts.loc[:, "RGI70ID"] = landsat_fronts["RGI60ID"].map(lambda x: RGI_link.loc[RGI_link.RGI60ID ==x].RGI70ID.values[0] if len(RGI_link.loc[RGI_link.RGI60ID ==x].RGI70ID.values) > 0 else None)
landsat_fronts.head()

,image,mission,timestamp,RGI60ID,geometry,RGI70ID
0,LM02_L1GS_245002_19760505_20200907_02_T2,Landsat 2 MSS,1976-05-05,RGI60-07.00791,"LINESTRING (12.38191 79.53251, 12.38144 79.531...",RGI2000-v7.0-G-07-01223
1,LM02_L1GS_245002_19760505_20200907_02_T2,Landsat 2 MSS,1976-05-05,RGI60-07.00777,"LINESTRING (12.38935 79.54895, 12.38853 79.549...",RGI2000-v7.0-G-07-01225
2,LM02_L1GS_245002_19760505_20200907_02_T2,Landsat 2 MSS,1976-05-05,RGI60-07.00775,"LINESTRING (12.32881 79.58411, 12.32937 79.585...",RGI2000-v7.0-G-07-01229
3,LM02_L1GS_245002_19760505_20200907_02_T2,Landsat 2 MSS,1976-05-05,RGI60-07.00794,"LINESTRING (12.29071 79.65163, 12.28778 79.651...",RGI2000-v7.0-G-07-01268
4,LM02_L1GS_245002_19760505_20200907_02_T2,Landsat 2 MSS,1976-05-05,RGI60-07.00790,"LINESTRING (12.1169 79.65285, 12.11207 79.6529...",RGI2000-v7.0-G-07-01266


In [4]:
# missing geometries
s = landsat_fronts.geometry.isna()
nonelist = s.where(s).dropna().index.to_list()

# landsat_fronts.iloc[nonelist]
landsat_fronts = landsat_fronts.drop(nonelist)

In [5]:
landsat_fronts["year"] = landsat_fronts["timestamp"].map(lambda x: x.year)

In [6]:
landsat_fronts = landsat_fronts.to_crs(epsg=32633)
landsat_fronts.crs

<Projected CRS: EPSG:32633>
Name: WGS 84 / UTM zone 33N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 12°E and 18°E, northern hemisphere between equator and 84°N, onshore and offshore. Austria. Bosnia and Herzegovina. Cameroon. Central African Republic. Chad. Congo. Croatia. Czechia. Democratic Republic of the Congo (Zaire). Gabon. Germany. Hungary. Italy. Libya. Malta. Niger. Nigeria. Norway. Poland. San Marino. Slovakia. Slovenia. Svalbard. Sweden. Vatican City State.
- bounds: (12.0, 0.0, 18.0, 84.0)
Coordinate Operation:
- name: UTM zone 33N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [7]:
# TODO: save final dataset

# Accuracies of Landsat Fronts

__Objective :__ calculate the distances between the fronts measured during the same year (when several fronts are available) to assess accuracy of georeferencing + manual delineation.

## Method 1 : calculate distances between the fronts themselves.
Doesn't work because the fronts don't necessarily have the same spatial extent.

In [ ]:
# glaciers present in landsat pictures
landsat_glaciers = landsat_fronts.RGIID.value_counts().index.to_list()

# accuracies of outlines on landsat pictures
accuracies = pd.DataFrame(
    {"RGIID": [],
     "distance_max_1978": [],
     "distance_max_1976": []}
)

for i_g, glacier in enumerate(landsat_glaciers):
    fronts_1978 = landsat_fronts.where(landsat_fronts.RGIID == glacier).where(landsat_fronts.year == 1978).dropna()
    fronts_1976 = landsat_fronts.where(landsat_fronts.RGIID == glacier).where(landsat_fronts.year == 1976).dropna()
    
    distances_1978 = []
    for i1 in range(len(fronts_1978)):
        f1 = fronts_1978.iloc[i1]
        for i2 in range(len(fronts_1978)):
            f2 = fronts_1978.iloc[i2]
            if i1 != i2:
                d = shapely.frechet_distance(f1.geometry, f2.geometry)
                distances_1978.append(d)
                
    distances_1976 = []
    for i1 in range(len(fronts_1976)):
        f1 = fronts_1976.iloc[i1]
        for i2 in range(len(fronts_1976)):
            f2 = fronts_1976.iloc[i2]
            if i1 != i2:
                d = shapely.frechet_distance(f1.geometry, f2.geometry)
                distances_1976.append(d)
                
    distance_max_1978 = max(distances_1978) if len(distances_1978) > 0 else None
    distance_max_1976 = max(distances_1976) if len(distances_1976) > 0 else None
    
    accuracies.loc[i_g] = [glacier, distance_max_1978, distance_max_1976]
    
# glaciers with multiple fronts on at least 1 year & multiple calving fronts.
# TODO: include these glaciers in the uncertainty assessment.
multiple_fronts_glaciers = [
    'RGI60-07.01482', 
    'RGI60-07.00854', 
    'RGI60-07.01510', 
    'RGI60-07.01449', 
    'RGI60-07.00888', 
    'RGI60-07.01534',
    'RGI60-07.01354',
    'RGI60-07.00030',
    'RGI60-07.00035',
    'RGI60-07.00984',
    'RGI60-07.00558',
    'RGI60-07.01523',
    'RGI60-07.01470'
]

# remove nan values and glaciers with multiple fronts
indexes_to_drop = [i for i in range(len(accuracies)) if ((accuracies.distance_max_1978.isna().iloc[i]) and (accuracies.distance_max_1976.isna().iloc[i])) or (accuracies.RGIID.iloc[i] in multiple_fronts_glaciers)]
accuracies = accuracies.drop(indexes_to_drop)

accuracies

In [ ]:
accuracies.sort_values(by="distance_max_1978", ascending=False)

In [ ]:
# some statistics (evaluate the precision of the georeferencing: need to add uncertainty due to the images resolution)
# Re: the glaciers can still be moving between the two measurements (ie. surge)
print("mean, std, min, max, count")
print(accuracies.distance_max_1978.mean(), accuracies.distance_max_1978.std(), accuracies.distance_max_1978.min(), accuracies.distance_max_1978.max(), len(accuracies.distance_max_1978.where(accuracies.distance_max_1978.isna() == False).dropna()))
print(accuracies.distance_max_1976.mean(), accuracies.distance_max_1976.std(), accuracies.distance_max_1976.min(), accuracies.distance_max_1976.max(), len(accuracies.distance_max_1976.where(accuracies.distance_max_1976.isna() == False).dropna()))

## Method 2 : calculate distances between front points and centerlines
Centerlines available in RGIv7 → necessary to load RGIv7 indexes (cf. Load ad postprocess data)

In [8]:
# load centerlines
centerlines = gpd.read_file(os.path.join(data_root, "Randolph_glacier_inventory/rgi70_svalbard/RGI2000-v7.0-L-07_svalbard_jan_mayen/RGI2000-v7.0-L-07_svalbard_jan_mayen.shp"))
centerlines.head()

,rgi_id,rgi_g_id,segment_id,is_main,outflow_id,strahler_n,length_m,geometry
0,RGI2000-v7.0-L-07-00001,RGI2000-v7.0-G-07-00001,0,1,-1,1,1616,"LINESTRING (10.76008 79.54671, 10.76004 79.546..."
1,RGI2000-v7.0-L-07-00002,RGI2000-v7.0-G-07-00002,0,0,1,1,989,"LINESTRING (10.81281 79.5405, 10.81282 79.5405..."
2,RGI2000-v7.0-L-07-00003,RGI2000-v7.0-G-07-00002,1,0,2,2,1537,"LINESTRING (10.85019 79.5428, 10.85014 79.5429..."
3,RGI2000-v7.0-L-07-00004,RGI2000-v7.0-G-07-00002,2,1,-1,3,2290,"LINESTRING (10.76824 79.54018, 10.76883 79.540..."
4,RGI2000-v7.0-L-07-00005,RGI2000-v7.0-G-07-00003,0,1,-1,1,914,"LINESTRING (10.88164 79.54685, 10.88167 79.546..."


In [42]:
centerlines = centerlines.to_crs(epsg=32633)
centerlines.crs

<Projected CRS: EPSG:32633>
Name: WGS 84 / UTM zone 33N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 12°E and 18°E, northern hemisphere between equator and 84°N, onshore and offshore. Austria. Bosnia and Herzegovina. Cameroon. Central African Republic. Chad. Congo. Croatia. Czechia. Democratic Republic of the Congo (Zaire). Gabon. Germany. Hungary. Italy. Libya. Malta. Niger. Nigeria. Norway. Poland. San Marino. Slovakia. Slovenia. Svalbard. Sweden. Vatican City State.
- bounds: (12.0, 0.0, 18.0, 84.0)
Coordinate Operation:
- name: UTM zone 33N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
for i in landsat_fronts.index.values:
    RGI70ID = landsat_fronts.loc[i, "RGI70ID"]

    geometry = landsat_fronts.loc[i, "geometry"] # front geometry
    references = centerlines.loc[centerlines.rgi_g_id == RGI70ID, "geometry"].values # all centerlines for this glacier
    nearests = [utils.geometry.extract_nearest(geometry, r) for r in references] # nearsest point of front for each centerline
    
    _, p, _ = min(nearests, key=lambda x: x[2]) # closest point
    distances = np.array([x[2] for x in nearests])
    j = distances.argmin() # index of the closest centerline

    landsat_fronts.loc[i, "centerline"] = centerlines.loc[centerlines.rgi_g_id == RGI70ID, "rgi_id"].values[j]
    landsat_fronts.loc[i, "point_centerline"] = p
    
landsat_fronts.head()

,image,mission,timestamp,RGI60ID,geometry,RGI70ID,year,centerline,point_centerline
0,LM02_L1GS_245002_19760505_20200907_02_T2,Landsat 2 MSS,1976-05-05,RGI60-07.00791,"LINESTRING (446917.357 8830600.499, 446900.231...",RGI2000-v7.0-G-07-01223,1976,RGI2000-v7.0-L-07-03355,POINT (447270.1670913332 8829908.581419634)
1,LM02_L1GS_245002_19760505_20200907_02_T2,Landsat 2 MSS,1976-05-05,RGI60-07.00777,"LINESTRING (447150.28 8832426.204, 447136.579 ...",RGI2000-v7.0-G-07-01225,1976,RGI2000-v7.0-L-07-03362,POINT (447150.2803240092 8832426.203533435)
2,LM02_L1GS_245002_19760505_20200907_02_T2,Landsat 2 MSS,1976-05-05,RGI60-07.00775,"LINESTRING (446105.553 8836403.019, 446122.679...",RGI2000-v7.0-G-07-01229,1976,RGI2000-v7.0-L-07-03368,POINT (446465.21308215824 8837012.728717625)
3,LM02_L1GS_245002_19760505_20200907_02_T2,Landsat 2 MSS,1976-05-05,RGI60-07.00794,"LINESTRING (445687.662 8843966.161, 445629.431...",RGI2000-v7.0-G-07-01268,1976,RGI2000-v7.0-L-07-03458,POINT (444478.5180807904 8844020.966601761)
4,LM02_L1GS_245002_19760505_20200907_02_T2,Landsat 2 MSS,1976-05-05,RGI60-07.00790,"LINESTRING (442212.658 8844269.303, 442116.749...",RGI2000-v7.0-G-07-01266,1976,RGI2000-v7.0-L-07-03449,POINT (442116.748764509 8844289.855494188)


In [45]:
landsat_fronts.loc[:, ["timestamp", "mission", "image", "RGI60ID", "RGI70ID", "centerline", "geometry"]].to_file(os.path.join(fronts_root, "Landsat_MSS_fronts_2.shp"), driver="ESRI Shapefile")

In [46]:
landsat_fronts_centerlines = landsat_fronts.loc[:, ["timestamp", "mission", "image", "RGI60ID", "RGI70ID", "centerline", "geometry"]]
landsat_fronts_centerlines.loc[:, "geometry"] = landsat_fronts["point_centerline"]
landsat_fronts_centerlines.to_file(os.path.join(fronts_root, "Landsat_MSS_fronts_2_centerline_points.shp"), driver="ESRI Shapefile")

This method works in most cases (some centerline confusions).
BUT: the produced points are taken from the geometries, an interpolation would give a more accurate result.

# Load Li fronts

In [ ]:
li_root = "D:/OneDrive/Documents/Cours/4A/SFE/data/Li_et_al_2023_Svalbard_v3"
li_fronts = gpd.read_file(os.path.join(li_root, "Svalbard_Calving_Front_Product.gpkg"), layer="traces").drop(columns=["year"])
li_fronts.columns = ["RGIID", "Sensor", "ImageId", "date", "CFL_Change", "geometry"]

# add timestamp
timestamp = []
for i in range(len(li_fronts)):
    date_i = li_fronts["date"].iloc[i]
    timestamp_i = datetime.datetime(int(date_i[0:4]), int(date_i[4:6]), int(date_i[6:8]))
    timestamp.append(timestamp_i)
li_fronts["timestamp"] = timestamp
li_fronts

In [ ]:
li_fronts["year"] = li_fronts["timestamp"].map(lambda x: x.year)

In [ ]:
li_fronts = li_fronts.to_crs(epsg=32633)
li_fronts.crs

In [ ]:
li_glaciers = li_fronts.RGIID.value_counts().index
li_years = li_fronts.year.value_counts().index.to_list()

# li_data_availability = pd.DataFrame(columns=li_years)
# for i_g, glacier in enumerate(li_glaciers):
#     li_data_availability.loc[i_g, "RGIID"] = glacier
#     for i_y, year in enumerate(li_years):
#         li_fronts_g_y = li_fronts.where(li_fronts.RGIID == glacier).where(li_fronts.year == year).dropna()
#         li_data_availability.loc[i_g, year] = len(li_fronts_g_y)
        
# li_data_availability.to_csv(os.path.join(li_root, "li_data_availability.csv"), index=False)   
li_data_availability = pd.read_csv(os.path.join(li_root, "li_data_availability.csv"))
li_data_availability

# Compare fronts

In [ ]:
comparable_glaciers = []
for id in li_glaciers:
    if id in landsat_glaciers:
        comparable_glaciers.append(id)
        
len(comparable_glaciers)

In [ ]:
for i_g, glacier in enumerate(comparable_glaciers):
    fronts_1978 = landsat_fronts.where(landsat_fronts.RGIID == glacier).where(landsat_fronts.year == 1978).dropna()
    fronts_1976 = landsat_fronts.where(landsat_fronts.RGIID == glacier).where(landsat_fronts.year == 1976).dropna()
    
    for i_y, year in enumerate(li_years):
        distances_1978 = []
        for i1 in range(len(fronts_1978)):
            f1 = fronts_1978.iloc[i1]
            for i2 in range(len(fronts_1978)):
                f2 = fronts_1978.iloc[i2]
                if i1 != i2:
                    d = shapely.distance(f1.geometry, f2.geometry)
                    distances_1978.append(d)
                
    distances_1976 = []
    for i1 in range(len(fronts_1976)):
        f1 = fronts_1976.iloc[i1]
        for i2 in range(len(fronts_1976)):
            f2 = fronts_1976.iloc[i2]
            if i1 != i2:
                d = shapely.distance(f1.geometry, f2.geometry)
                distances_1976.append(d)
                
    distance_max_1978 = max(distances_1978) if len(distances_1978) > 0 else None
    distance_max_1976 = max(distances_1976) if len(distances_1976) > 0 else None
    
    accuracies.loc[i_g] = [glacier, distance_max_1978, distance_max_1976]
    
# glaciers with multiple fronts on at least 1 year & multiple calving fronts.
# TODO: include these glaciers in the uncertainty assessment.
multiple_fronts_glaciers = [
    'RGI60-07.01482', 
    'RGI60-07.00854', 
    'RGI60-07.01510', 
    'RGI60-07.01449', 
    'RGI60-07.00888', 
    'RGI60-07.01534',
    'RGI60-07.01354',
    'RGI60-07.00030',
    'RGI60-07.00035',
    'RGI60-07.00984',
    'RGI60-07.00558',
    'RGI60-07.01523',
    'RGI60-07.01470'
    
]

# remove nan values and glaciers with multiple fronts
indexes_to_drop = [i for i in range(len(accuracies)) if ((accuracies.distance_max_1978.isna().iloc[i]) and (accuracies.distance_max_1976.isna().iloc[i])) or (accuracies.RGIID.iloc[i] in multiple_fronts_glaciers)]
accuracies = accuracies.drop(indexes_to_drop)

accuracies

In [ ]:
landsat_fronts.where(landsat_fronts.RGIID.isin(comparable_glaciers)).dropna()